# Auxiliary.ipynb

#### Notebook containing computations that multiple data analysis notebooks need, such as waveform reading, surrogate evaluation, overlap computation, and SNR computation

Maria Okounkova (mokounkova@flatironinstitute.org)

In [15]:
import matplotlib.pyplot as plt
import h5py
from astropy import constants as const
import seaborn as sns
import numpy as np
from math import pi
import matplotlib
from scipy.interpolate import InterpolatedUnivariateSpline
import gwsurrogate
from pycbc.detector import Detector
import pycbc
from pycbc.filter.matchedfilter import overlap

### General auxiliary methods - subtracting peak times, computing $\Delta t$, ramp function, etc

In [2]:
def GetPeakTime(time, data): 
    """ Grab the peak time of some data """
    t_peak = time[np.argmax(data)]
    return t_peak

def SubtractPeakTime(time, data): 
    """ Subtract the peak time of some data """
    t_peak = GetPeakTime(time, data)
    return time - t_peak

def dt_eval(time):
    """ Return the time step of a given time array """
    return (time[1] - time[0])

def df_eval(time):
    """ Return the delta_f of a given time array """
    delta_t = dt_eval(time)
    return 1.0/((time[-1] - time[0]) + delta_t)

def Ramp(time, t_s, t_r):
    """ Ramp function for tapering the waveform"""
    if (time < t_s):
        return 0.0
    elif time > (t_s + t_r):
        return 1.0
    else:
        t = (time - t_s)/t_r
        return t**5*(126 + t*(-420 + t*(540 + t*(-315 + 70*t))))

### dCS auxiliary methods - converting dimensionless coupling constant to kms, etc

In [3]:
def EllinKm(ell, mass):
    """ Return the value of the dCS coupling constant in km """
    if 'p' in ell:
        ## If we're using a string like 0p0 for 0.0, convert to a float
        ell = float(ell.replace('p', '.'))
    mass_msun = mass * const.M_sun
    phys_ell_km = ell * mass_msun * const.G /(const.c**2) / 1000
    return phys_ell_km.value

def EllString(ell):
    return str(ell).replace('.','p')

### NR auxiliary methods - waveform reading methods

In [4]:
def swsh(s, modes, theta, phi, psi=0):
    """
    Return a value of a spin-weighted spherical harmonic of spin-weight s. 
    If passed a list of several modes, then a numpy array is returned with 
    SWSH values of each mode for the given point.
    For one mode:       swsh(s,[(l,m)],theta,phi,psi=0)
    For several modes:  swsh(s,[(l1,m1),(l2,m2),(l3,m3),...],theta,phi,psi=0)
    """
    import spherical_functions as sf
    import quaternion as qt
    return sf.SWSH(qt.from_spherical_coords(theta, phi), s, modes) * np.exp(1j * s * psi)

In [5]:
def ReadExtrapolatedModes(file, params_dict, interpolate = True):
    """ 
        File is the file containing the extrapolated waveform that we want to read in.
        For params_dict, 
        mass_msun is the total mass of the system in solar masses, and 
        dist_mpc is the distance to the system in kpc. 
        theta and phi are angles determining the inclination.
        dt is the timestep (reciprocal of the sampling rate)
        
        If we want to interpolate the waveform to have even timesteps dt, then 
        set interpolate to True. Otherwise, we'll return the data without 
        performing the interpolation 

        """

    ## Convert distance to kpc and mass into solar masses
    mass = params_dict['mass']
    dist_mpc = params_dict['dist_mpc']
    theta = params_dict['theta']
    phi = params_dict['phi'] 
    dt = params_dict['dt']
    dist_kpc = dist_mpc * 1000 * const.kpc
    mass_msun = mass * const.M_sun
    
    ## Read in the data
    f = h5py.File(file, 'r')
    
    ## grab the length of the waveform first
    data = f['Extrapolated_N2.dir']['Y_l2_m2.dat']
    time = np.array(data[:,0])
    
    h_plus = np.zeros(len(time))
    h_cross = np.zeros(len(time))

    modes = [(l,m) for l in range(2,5) for m in range(-l, l+1)]
    for mode in modes: 
        
        ## Grab the mode in question
        l = mode[0]
        m = mode[1]
        
        data = f['Extrapolated_N2.dir']['Y_l' + str(l) + '_m' + str(m) + '.dat']
        real = np.array(data[:,1])
        imag = np.array(data[:,2])
        coeff = real + 1j * imag
        
        ## Multiply by the corresponding spin-weighted spherical harmonic
        Ylm = swsh(-2, [(l,m)], theta=theta, phi=phi, psi=0) 
        h = coeff * Ylm 
        
        ## Add to our h_plus and h_cross computations
        h_plus = h_plus + np.real(h)
        h_cross = h_cross - np.imag(h) 
        
    ## Apply the astrophysical parameters
    time = time*mass_msun*const.G/(const.c**3)
    h_plus = h_plus*const.G*mass_msun/((const.c)**2*dist_kpc)
    h_cross = h_cross*const.G*mass_msun/((const.c)**2*dist_kpc)

    ## Taper the waveform and apply the ramp (need to start the waveform at zero for this)
    time = time - time[0]
    ramp = np.array([Ramp(t.value, 0.1, 0.3) for t in time])
    
    h_plus = h_plus * ramp
    h_cross = h_cross * ramp
    
    ## Now subtract off the peak time (this makes the spline interpolation easier)
    amp = np.sqrt(h_plus**2 + h_cross**2)
    time = time - time[np.argmax(amp)]
    
    if not interpolate:
        print("Not performing the interpolation")
        return time, h_plus, h_cross, np.sqrt(h_plus**2 + h_cross**2)
    
    ## Now build the interpolants 
    cs_plus = InterpolatedUnivariateSpline(time, h_plus)
    cs_cross = InterpolatedUnivariateSpline(time, h_cross)

    ## Now create an evenly-spaced time array and interpolate the data 
    time_cs = np.arange(time[0].value, time[-1].value, dt)

    h_plus_cs = cs_plus(time_cs) 
    h_cross_cs = cs_cross(time_cs) 
    
    ## Return these new interpolated values
    return time_cs, h_plus_cs, h_cross_cs, np.sqrt(h_plus_cs**2 + h_cross_cs**2)

### Surrogate auxiliary methods - surrogate model evaluation

In [18]:
def EvaluateSurrogate(sur, params_dict):
    """ Evaluate the surrogate waveform using a dictionary of parameters
        and an instance of a surrogate model """

    
    phi_ref = pi/2 - params_dict['phi']
    #print('Surrogate phase evaluation: ', phi_ref), 
    data = sur(params_dict['q'], params_dict['a_1'], params_dict['a_2'], \
               dt = params_dict['dt'], units = 'mks', M = params_dict['mass'], \
               dist_mpc = params_dict['dist_mpc'], f_low = params_dict['f_low'], \
               inclination = params_dict['theta'], ellMax = 4, \
               phi_ref = phi_ref)

    time = np.array(data[0])
    h_plus = np.real(data[1])
    h_cross = -1 * np.imag(data[1])

    ## Taper the waveform and apply the ramp (need to start the waveform at zero for this)
    #time = time - time[0]
    #ramp = np.array([Ramp(t, 0.1, 0.3) for t in time])
    
    #h_plus = h_plus * ramp
    #h_cross = h_cross * ramp
    
    ## Subtract off the peak time of the waveform
    amp = np.sqrt(h_plus**2 + h_cross**2)
    time = time - time[np.argmax(amp)]
    
    return time, h_plus, h_cross, np.sqrt(h_plus**2 + h_cross**2)

### Detector auxiliary methods - $h_\times$ and $h_\times$ to detectors, padding time segments

In [ ]:
def PadAndProject(time, h_plus, h_cross, params_dict, geocent_time = False):
    """ TODO: documentation here """
    
    dt = params_dict['dt']
    ## If we have geocenter_time, then correct this to get the t0
    
    ## Let t0 be the gps time at which the signal arrives at Hanford
    t0 = params_dict['t_gps']
    ra = params_dict['ra']
    dec = params_dict['dec']
    pol = params_dict['pol']
    peak_time_in_segment = params_dict['peak_time_in_segment']
    segment_length = params_dict['segment_length']
    
    print("t0 parameter: ", t0)
    
    ## h_plus and h_cross start out with the same time since we haven't 
    ## done any detector projection -- the signal reaches both the interferometers --
    ## we just have to figure out when this is
    d_H1 = Detector("H1")
    d_L1 = Detector("L1")
    
    ## Time delay between the detectors
    t_delay_LH = d_L1.time_delay_from_detector(d_H1, ra, dec, t0)
    print("Time delay between detectors", t_delay_LH)
    ## Antenna Patterns
    Fp_H1, Fc_H1 = d_H1.antenna_pattern(ra, dec, pol, t0)
    Fp_L1, Fc_L1 = d_L1.antenna_pattern(ra, dec, pol, t0 + t_delay_LH)
    
    ## Then project to the detectors
    h_H1 = Fp_H1*h_plus + Fc_H1*h_cross
    h_L1 = Fp_L1*h_plus + Fc_L1*h_cross
    
    ## Now let's find the peak time in h_H1
    t_peak_in_H_initial = GetPeakTime(time, h_H1).value
    print("Peak time in H initially:", t_peak_in_H_initial)
    
    ## Shift the times so that the peak time in H occurs at t0
    time_H1 = np.array([t.value for t in time]) - t_peak_in_H_initial + t0
    time_L1 = np.array(time_H1) + t_delay_LH
    
    time_H1_peak = GetPeakTime(time_H1, h_H1)
    time_L1_peak = GetPeakTime(time_L1, h_L1)

    print("Peak time in H updated:", time_H1_peak)
    print("Peak time in L updated:", time_L1_peak)
    
    ## Now let's pad with zeroes before we do the interpolation
    segment_start_time = t0 - peak_time_in_segment
    segment_end_time = segment_start_time + segment_length
    print("Segment start and end: ", segment_start_time, segment_end_time)
    
    ## Make the time array 
    times_to_interpolate_to = np.arange(segment_start_time, segment_end_time, step = dt)
    
    ## Ensure that t0 is in the time array
    print("t0 is in the time array: ", t0 in times_to_interpolate_to)
    
    ## Pad the h_H1 and h_L1 arrays with zeros before doing the interpolation onto times_to_interpolate_to
    ## Roughly how much we want to bad one each side 
    number_of_pads = int(peak_time_in_segment / dt)
    
    h_H1_padded = np.pad(h_H1, (number_of_pads, number_of_pads), 'constant', constant_values=(0.0, 0.0))
    h_L1_padded = np.pad(h_L1, (number_of_pads, number_of_pads), 'constant', constant_values=(0.0, 0.0))
    
    time_H1_start_pad = time_H1[0] -dt - dt * np.array(range(number_of_pads))[::-1]
    time_H1_end_pad = time_H1[-1] + dt + dt * np.array(range(number_of_pads))
    
    time_L1_start_pad = time_L1[0] -dt - dt * np.array(range(number_of_pads))[::-1]
    time_L1_end_pad = time_L1[-1] + dt + dt * np.array(range(number_of_pads))
    
    time_H1_padded = np.concatenate((time_H1_start_pad, time_H1))
    time_H1_padded = np.concatenate((time_H1_padded, time_H1_end_pad))
    
    time_L1_padded = np.concatenate((time_L1_start_pad, time_L1))
    time_L1_padded = np.concatenate((time_L1_padded, time_L1_end_pad))

    ## Interpolate onto the time array
    cs_H1 = InterpolatedUnivariateSpline(time_H1_padded, h_H1_padded)
    cs_L1 = InterpolatedUnivariateSpline(time_L1_padded, h_L1_padded)
    
    h_H1_interpolated = cs_H1(times_to_interpolate_to) 
    h_L1_interpolated = cs_L1(times_to_interpolate_to) 
    
    time_H1_peak = GetPeakTime(time_H1, h_H1)
    time_L1_peak = GetPeakTime(time_L1, h_L1)

    print("Peak time in H after interpolation:", GetPeakTime(times_to_interpolate_to, h_H1_interpolated))
    print("Peak time in L after interpolation:", GetPeakTime(times_to_interpolate_to, h_L1_interpolated))
    
    ## Plotting if we want to check
#     plt.figure(figsize=(12, 8))
    
#     plt.plot(time_H1_padded, h_H1_padded)
#     plt.plot(time_H1, h_H1, '--', color = 'blue')
#     plt.plot(times_to_interpolate_to, h_H1_interpolated, alpha = 0.5, label = 'H1')
    
#     plt.plot(time_L1_padded, h_L1_padded)
#     plt.plot(time_L1, h_L1, '--', color = 'black')
#     plt.plot(times_to_interpolate_to, h_L1_interpolated, alpha = 0.5, label = 'L1')
    
#     plt.xlim(t0 - 0.3, t0 + 0.1)
#     plt.legend()
#     plt.show()
    
    return times_to_interpolate_to, h_H1_interpolated, times_to_interpolate_to, h_L1_interpolated

### Detector auxiliary methods - $h_\times$ and $h_\times$ to detectors given a geocenter time

In [16]:
def ProjectGivenGeocenterTime(time, h_plus, h_cross, params_dict):
    """ Given an h_plus and h_cross, and a dictionary of parameters with a 
        geocenter time, project onto the gravitational wave detectors """
    
    ## Set up the detectors we'll be using
    d_H1 = Detector("H1")
    d_L1 = Detector("L1")
    
    ## Obtain the geocenter_time from the Bilby search results
    geocent_time = params_dict['geocent_time']
    
    ## Grab source location and polarization parameters from the Bilby search results
    ra = params_dict['ra']
    dec = params_dict['dec']
    pol = params_dict['pol']
    
    ## Now we want to compute t_gps_H and t_gps_L, the gps time at each detector.
    ## We need this in order to specify the antenna pattern, and also to shift
    ## the time arrays. 
    
    ## Use time_delay_from_earth_center(right_ascension, declination, t_gps) from pycbc
    ## https://pycbc.org/pycbc/latest/html/pycbc.html?highlight=time_delay#pycbc.detector.Detector.time_delay_from_earth_center
    ## In our case, we have the geocenter time, and we want the gps time, so we can call the function as
    ## time_delay_from_earth_center(ra, dec, geocent_time), and then flip the sign of the result, 
    ## which we can do since ra, dec are time-independent for LIGO data.
    
    time_delay_H = d_H1.time_delay_from_earth_center(ra, dec, geocent_time)
    time_delay_L = d_L1.time_delay_from_earth_center(ra, dec, geocent_time)
    #print("time delay H: ", time_delay_H)
    #print("time delay L: ", time_delay_L)
    
    ## Add in the time delays, since they're the "extra time" (which can be negative)
    ## that the signal takes to travel from the center of the earth to the detectors
    t_gps_H = geocent_time + time_delay_H
    t_gps_L = geocent_time + time_delay_L
    #print("t_gps_H: ", t_gps_H)
    #print("t_gps_L: ", t_gps_L)
    
    ## Check that the time delay between H and L agrees with the pycbc time_delay_from_detector method
    ## https://pycbc.org/pycbc/latest/html/pycbc.html?highlight=time_delay#pycbc.detector.Detector.time_delay_from_detector
    t_delay_LH = d_L1.time_delay_from_detector(d_H1, ra, dec, t_gps_H)
    #print("Time delay LH: ", t_delay_LH)
    
    ## Compute the antenna patterns, which take the gps time as the argument
    ## See https://pycbc.org/pycbc/latest/html/pycbc.html?highlight=antenna_pattern#pycbc.detector.Detector.antenna_pattern
    Fp_H1, Fc_H1 = d_H1.antenna_pattern(ra, dec, pol, t_gps_H)
    Fp_L1, Fc_L1 = d_L1.antenna_pattern(ra, dec, pol, t_gps_L)
    
    ## Project the strain onto the detectors
    h_H1 = Fp_H1*h_plus + Fc_H1*h_cross
    h_L1 = Fp_L1*h_plus + Fc_L1*h_cross
    
    ## Shift the time array to the gps times in each detector. 
    ## When we call the surrogate model, we subtract np.sqrt(h_plus**2 + h_cross**2)
    ## off of the waveform, to that the `time` array is zero when the 
    ## injected signal has maximum amplitude
    time_H1 = time + t_gps_H
    time_L1 = time + t_gps_L

    return time_H1, h_H1, time_L1, h_L1

### SNR and Overlap Auxiliary methods - Computing single and multi detector overlaps, computing SNRs

Single-detector inner product computation with pycbc. For the unnormalized inner product, we assume that the pycbc overlap method uses a form such as 

$$
\langle{h_\mathrm{A}, h_\mathrm{B}\rangle} \equiv 4 \mathrm{Re} \int_0^\infty \frac{\tilde{h}_\mathrm{A}(f) \tilde{h}_\mathrm{B}^{*}(f)}{S (f)} df
$$

When normalized = True, we assume that pycbc overlap returns

$$
\mathcal{O}_{\mathrm{A},\mathrm{B}} \equiv \frac{\langle{h_\mathrm{A}, h_\mathrm{B}\rangle}}{\sqrt{\langle{h_\mathrm{A}, h_\mathrm{A}\rangle} \langle{h_\mathrm{B}, h_\mathrm{B}\rangle}}}
$$

and when normalize = False, we assume that pycbc overlap returns

$$
\langle{h_\mathrm{A}^n, h_\mathrm{B}^n\rangle}
$$


In [ ]:
def ComputeInnerProduct(time1, strain1, time2, strain2, normalized = False, psd = True):
    """ Given two time-domain strains and the corresponding time arrays, compute the 
        inner product in one detector using pycbc. 
        Use normalized = True for overlaps, and normalized = False for SNR
        Assuming that this is Eq. 4 in https://arxiv.org/abs/2003.09456
        If we want it normalized, then it's like Eq. 8 in https://arxiv.org/abs/2003.09456
        If psd = True, use the advanced LIGO design PSD. Otherwise, don't use a psd
    """

    ## Construct time array to be interpolated onto, since the pycbc overlap computation
    ## requires the time arrays to be the same
    delta_t = dt_eval(time1)
    time = np.arange(start = max(time1[0], time2[0]), stop = min(time1[-1], time2[-1]), step = delta_t)
    delta_f = df_eval(time)
    
    ## Interpolate strains onto time array
    cs1 = InterpolatedUnivariateSpline(time1, strain1)
    cs2 = InterpolatedUnivariateSpline(time2, strain2)
    
    strain1 = cs1(time) 
    strain2 = cs2(time)

    ## Interpolated PSD onto df-spaced values
    freqs = delta_f * np.array(range(len(time)))
    
    psd_fs = None
    
    if psd: 
        ## Read in PSD and construct interpolant
        psd_file = "PSDs/design/aLIGOZeroDetHighPower-PSD.txt"
        psd_frequencies, psd_vals = np.loadtxt(psd_file, comments="#",usecols=([0,1]),unpack=True)
        cs = InterpolatedUnivariateSpline(psd_frequencies, psd_vals)
        psd_interp = cs(freqs)
        psd_fs = pycbc.types.frequencyseries.FrequencySeries(psd_interp, delta_f = delta_f, epoch = time[0])

    ## Timeseries and Frequency series objects for pycbc computation
    strain1_ts = pycbc.types.timeseries.TimeSeries(strain1, delta_t = delta_t, epoch = time[0])
    strain2_ts = pycbc.types.timeseries.TimeSeries(strain2, delta_t = delta_t, epoch = time[0])
    
    overlap_val = overlap(vec1 = strain1_ts, vec2 = strain2_ts, psd = psd_fs, normalized = normalized, \
                          low_frequency_cutoff = 25, high_frequency_cutoff = 2048) 

    return overlap_val

similar but frequency domain inner product

In [ ]:
def ComputeInnerProductFrequency(delta_f, strain1, strain2, normalized = True, psd = True):
    """ Given two frequency-domain strains and the corresponding delta_f, compute the 
        inner product in one detector using pycbc. 
        Use normalized = True for overlaps, and normalized = False for SNR
        Assuming that this is Eq. 4 in https://arxiv.org/abs/2003.09456
        If we want it normalized, then it's like Eq. 8 in https://arxiv.org/abs/2003.09456
        If psd = True, use the advanced LIGO design PSD. Otherwise, don't use a psd
    """
    psd_fs = None
    
    if psd: 
        ## Read in PSD and construct interpolant
        psd_file = "PSDs/design/aLIGOZeroDetHighPower-PSD.txt"
        psd_frequencies, psd_vals = np.loadtxt(psd_file, comments="#",usecols=([0,1]),unpack=True)
        cs = InterpolatedUnivariateSpline(psd_frequencies, psd_vals)
        freqs = delta_f * np.array(range(len(strain1)))
        psd_interp = cs(freqs)
        psd_fs = pycbc.types.frequencyseries.FrequencySeries(psd_interp, delta_f = delta_f, epoch = None)
        
    strain1_fs = pycbc.types.frequencyseries.FrequencySeries(strain1, delta_f = delta_f, epoch = None)
    strain2_fs = pycbc.types.frequencyseries.FrequencySeries(strain2, delta_f = delta_f, epoch = None)
    
    overlap_val = overlap(vec1 = strain1_fs, vec2 = strain2_fs, psd = psd_fs, normalized = normalized, \
                          low_frequency_cutoff = 25, high_frequency_cutoff = 2048) 

    return overlap_val

Compute the overlap $\mathcal{O}_{\mathrm{A},\mathrm{B}}$ between two waveforms in one detector with the normalized inner product

In [ ]:
def ComputeOverlap(time1, strain1, time2, strain2, psd = True):
    """ Compute overlap in one detector - Eq. 8 in arxiv.org/abs/2003.09456
    """
    inner_product = ComputeInnerProduct(time1, strain1, time2, strain2, normalized = True, psd = psd)
    return inner_product

def ComputeOverlapFrequency(delta_f, strain1, strain2, psd = True):
    """ Compute overlap in one detector - Eq. 8 in arxiv.org/abs/2003.09456 in the frequency domain
    """
    inner_product = ComputeInnerProductFrequency(delta_f, strain1, strain2, normalized = True, psd = psd)
    return inner_product

Compute the inner product in multiple detectors, using

$$
\langle{h_\mathrm{A}, h_\mathrm{B}\rangle}_N = \sum_{n}^N \langle{h_\mathrm{A}^n, h_\mathrm{B}^n\rangle}
$$

where the sum runs over inner products in each detector

In [ ]:
def ComputeMultiDetectorInnerProduct(time1_H, strain1_H, time1_L, strain1_L, \
                                time2_H, strain2_H, time2_L, strain2_L):
    """ Given two strain waveforms in both H1 and L1, compute the multi-detector inner product.
        Use normalized = True for overlaps, and normalized = False for SNR.
        This is Eq. 3 in arxiv.org/abs/2003.09456
    """
    
    inner_H = ComputeInnerProduct(time1_H, strain1_H, time2_H, strain2_H, normalized = False)
    inner_L = ComputeInnerProduct(time1_L, strain1_L, time2_L, strain2_L, normalized = False)
    ## Sum the detector results
    return inner_H + inner_L

def ComputeMultiDetectorInnerProductFrequency(delta_f, strain1_H, strain1_L, strain2_H, strain2_L):
    """ Given two strain waveforms in both H1 and L1 in the frequency domain, 
        compute the multi-detector inner product.
        Use normalized = True for overlaps, and normalized = False for SNR.
        This is Eq. 3 in arxiv.org/abs/2003.09456
    """
    
    inner_H = ComputeInnerProductFrequency(delta_f, strain1_H, strain2_H, normalized = False)
    inner_L = ComputeInnerProductFrequency(delta_f, strain1_L, strain2_L, normalized = False)
    ## Sum the detector results
    return inner_H + inner_L

Compute the overlap in a network of detectors using

$$
\mathcal{O}_{\mathrm{A},\mathrm{B}} \equiv \frac{\langle{h_\mathrm{A}, h_\mathrm{B}\rangle}_N}{\sqrt{\langle{h_\mathrm{A}, h_\mathrm{A}\rangle}_N \langle{h_\mathrm{B}, h_\mathrm{B}\rangle}_N}}
$$

In [ ]:
def ComputeMultiDetectorOverlap(time1_H, strain1_H, time1_L, strain1_L, \
                                time2_H, strain2_H, time2_L, strain2_L):
    """ Compute the overlap between two two-detector signals using Eq. 8 of arxiv.org/abs/2003.09456 """
    
    ## numerator term
    O_AB = ComputeMultiDetectorInnerProduct(time1_H, strain1_H, time1_L, strain1_L, \
                                time2_H, strain2_H, time2_L, strain2_L)
    
    ## denominator terms
    O_AA = ComputeMultiDetectorInnerProduct(time1_H, strain1_H, time1_L, strain1_L, \
                                time1_H, strain1_H, time1_L, strain1_L)
    O_BB = ComputeMultiDetectorInnerProduct(time2_H, strain2_H, time2_L, strain2_L, \
                                time2_H, strain2_H, time2_L, strain2_L)
    
    result = O_AB / np.sqrt(O_AA * O_BB)
    return result

def ComputeMultiDetectorOverlapFrequency(delta_f, strain1_H, strain1_L, strain2_H, strain2_L):
    """ Compute the overlap between two two-detector signals using Eq. 8 of arxiv.org/abs/2003.09456
        in the frequency domain """
    
    ## numerator term
    O_AB = ComputeMultiDetectorInnerProductFrequency(delta_f, strain1_H, strain1_L, strain2_H, strain2_L)
    
    ## denominator terms
    O_AA = ComputeMultiDetectorInnerProductFrequency(delta_f, strain1_H, strain1_L, strain1_H, strain1_L)
    O_BB = ComputeMultiDetectorInnerProductFrequency(delta_f, strain2_H, strain2_L, strain2_H, strain2_L)
    
    result = O_AB / np.sqrt(O_AA * O_BB)
    return result

Compute a signal to noise ratio using

$$
\mathrm{SNR}^2 = \langle{h, h\rangle}_N
$$

for one signal $h$, (where we can sum over inner products if we're considering multiple detectors

In [ ]:
def ComputeSNR(time, strain):
    """ Given a time-domain strain and the corresponding time array, 
        compute the SNR using pycbc in one detector 
        Using Eq. 5 in arxiv.org/abs/2003.09456 """
    
    inner_product = ComputeInnerProduct(time, strain, time, strain, normalized = False)
    return np.sqrt(inner_product)


def ComputeMultiDetectorSNR(timeH, strainH, timeL, strainL):
    """ Compute an SNR in multiple detectors using Eq. 5 in arxiv.org/abs/2003.09456 
    """
    
    inner_product_H = ComputeInnerProduct(timeH, strainH, timeH, strainH, normalized = False)
    inner_product_L = ComputeInnerProduct(timeL, strainL, timeL, strainL, normalized = False)
    result = np.sqrt(inner_product_H + inner_product_L)
    return result

Given H and L gravitational wave strains along with a dictionary of parameters, adjust the distance to the source to that the signal has the desired SNR

In [1]:
def TargetSNR(timeH, strainH, timeL, strainL, params_dict, desired_snr):
    """ Update the distance for a given GW waveform in order to 
        achieve a target SNR in H1 [note that (SNR_H1 / SNR_L1) is 
        independent of the distance, and SNR scales with distance as
        SNR ~ 1 / distance
        """
    ## Compute the SNR as it stands
    current_snr = ComputeMultiDetectorSNR(timeH, strainH, timeL, strainL)
    print('Current SNR ', current_snr)
    
    ## Grab the current distance
    current_dist = params_dict['dist_mpc']
    print('Current distance', current_dist)
    
    ## Update the current distance to achieve the desired snr
    updated_dist = current_dist * current_snr / desired_snr
    print('Updated distance', updated_dist)
    
    ## Update the distance in the waveforms 
    strainH = strainH * current_dist / updated_dist
    strainL = strainL * current_dist / updated_dist

    ## Update the distance in the dictionary
    params_dict['dist_mpc'] = updated_dist
    
    return strainH, strainL, params_dict